In [ ]:
import os, glob, yaml, random, cv2, gc, torch
from ultralytics import YOLO

SEED = 321 # 123, 999
BASE = r"C:\Users\Think\Desktop\NesvarbuSefoSpecialusHomemade"
REAL = os.path.join(BASE, r"Datasets\UAV 1_Resized_512")
SYNTH_DIRS = [
    os.path.join(BASE, r"Datasets\GeneratedImagesEimis"),
    os.path.join(BASE, r"Datasets\GeneratedImagesVilius"),
    os.path.join(BASE, r"Datasets\GeneratedImagesJokubas")
]
SPLIT_DIR = os.path.join(BASE, f"app\Final_Data_Splits_Seed{SEED}")
RUNS_DIR = os.path.join(BASE, f"app\Final_Training_Runs_Seed{SEED}")
os.makedirs(SPLIT_DIR, exist_ok=True)

# aug
NO_AUG = {'hsv_h': 0, 'hsv_s': 0, 'hsv_v': 0, 'degrees': 0, 'translate': 0, 'scale': 0, 'shear': 0, 
          'perspective': 0, 'flipud': 0, 'fliplr': 0, 'mosaic': 0, 'mixup': 0, 'copy_paste': 0, 'erasing': 0}
HEAVY_AUG = {'degrees': 45, 'translate': 0.3, 'scale': 0.8, 'shear': 5, 'flipud': 0.5, 'fliplr': 0.5, 
             'mosaic': 1.0, 'mixup': 0.2, 'copy_paste': 0.3, 'erasing': 0.4, 'hsv_h': 0.05, 'hsv_s': 0.8, 'hsv_v': 0.6}

# DATA PREP 
def get_syn(dirs):
    valid = []
    for d in dirs:
        md = os.path.join(d, "GeneratedMasks")
        for p in glob.glob(os.path.join(d, "GeneratedImages", "*.*")):
            t = os.path.splitext(p)[0] + ".txt"
            if os.path.exists(t) and os.path.getsize(t) > 0: valid.append(p); continue
            m = os.path.join(md, os.path.basename(p).replace(os.path.splitext(p)[1], ".png"))
            if not os.path.exists(m): continue
            msk, img = cv2.imread(m, 0), cv2.imread(p)
            if msk is None or img is None: continue
            cts, _ = cv2.findContours(msk, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if not cts: continue
            lines = []
            h, w = img.shape[:2]
            for c in cts:
                if cv2.contourArea(c) > 10:
                    x, y, bw, bh = cv2.boundingRect(c)
                    lines.append(f"0 {(x+bw/2)/w:.6f} {(y+bh/2)/h:.6f} {bw/w:.6f} {bh/h:.6f}")
            if not lines: continue
            with open(t, "w") as f: f.write("\n".join(lines))
            valid.append(p)
    return valid

real = [os.path.abspath(p) for p in glob.glob(os.path.join(REAL, "images", "train", "*.*"))]
syn = get_syn(SYNTH_DIRS)
with open(os.path.join(SPLIT_DIR, "val.txt"), "w") as f: f.write("\n".join([os.path.abspath(p) for p in glob.glob(os.path.join(REAL, "images", "valid", "*.*"))]))

def make_cfg(name, files):
    with open(os.path.join(SPLIT_DIR, f"{name}.txt"), "w") as f: f.write("\n".join(files))
    with open(os.path.join(SPLIT_DIR, f"{name}.yaml"), "w") as f: yaml.dump({'path': SPLIT_DIR, 'train': f"{name}.txt", 'val': "val.txt", 'names': {0: 'building'}}, f)

N = len(real)
random.seed(SEED)
make_cfg("real100_only", random.sample(real, N))
make_cfg("multi_syn100_real0", random.sample(syn, N))

for p in [25, 50, 75]:
    nr, ns = int(N * p/100), N - int(N * p/100)
    random.seed(SEED); sub_r = random.sample(real, nr)
    make_cfg(f"real{p}_only", sub_r)
    random.seed(SEED); make_cfg(f"multi_syn{100-p}_real{p}", sub_r + random.sample(syn, ns))

runs = [
    ("real100_only", "real100_only", {}),
    ("real100_only", "yolo11s_2_default_aug", {}),
    ("real100_only", "yolo11s_3_heavy_aug", HEAVY_AUG),
    ("real100_only", "yolo11s_1_no_aug", NO_AUG),
    ("real75_only", "real75_only", {}),
    ("real50_only", "real50_only", {}),
    ("real25_only", "real25_only", {}),
    ("multi_syn25_real75", "multi_syn25_real75", NO_AUG),
    ("multi_syn50_real50", "multi_syn50_real50", NO_AUG),
    ("multi_syn75_real25", "multi_syn75_real25", NO_AUG),
    ("multi_syn100_real0", f"yolo11s_multi_syn100_real0_seed{SEED}", NO_AUG)
]

for cfg, name, kw in runs:
    if "seed" not in name and "yolo" not in name: name += f"_seed{SEED}"
    if os.path.exists(os.path.join(RUNS_DIR, name, "weights", "best.pt")): 
        print(f"Skipping {name}"); continue
    
    print(f"Training {name}...")
    try:
        model = YOLO("yolo11s.pt")
        model.train(
            data=os.path.join(SPLIT_DIR, f"{cfg}.yaml"), 
            project=RUNS_DIR, name=name, 
            epochs=100, 
            imgsz=640, 
            batch=-1,       
            workers=2,      
            cache=True,     
            val=False,      
            plots=False, 
            seed=SEED, **kw)
    except Exception as e: print(f"Error {name}: {e}")
    finally: del model; gc.collect(); torch.cuda.empty_cache()

In [ ]:
import os, json, shutil, pandas as pd, gc, torch
from tqdm import tqdm
from ultralytics import YOLO

SEED = 321 # 123, 999
BASE = r"C:\Users\Think\Desktop\NesvarbuSefoSpecialusHomemade"
REAL_VAL = os.path.join(BASE, r"Datasets\UAV 1_Resized_512\dataset.yaml")
RWDS_ROOT = r"C:\Users\Think\Desktop\RWDS\RWDS_Dataset\RWDS_Dataset\RWDS_FR\train"
OUT_RWDS = r"C:\Users\Think\Desktop\RWDS_Flooding_YOLO"
RUNS_DIR = os.path.join(BASE, f"app\Final_Training_Runs_Seed{SEED}")
EXCEL_PATH = os.path.join(BASE, f"app\Results_Seed{SEED}.xlsx")

def prep_rwds():
    os.makedirs(os.path.join(OUT_RWDS, "images"), exist_ok=True)
    os.makedirs(os.path.join(OUT_RWDS, "labels"), exist_ok=True)
    yaml_p = os.path.join(OUT_RWDS, "rwds.yaml")
    if not os.path.exists(yaml_p):
        with open(yaml_p, "w") as f: f.write(f"path: {OUT_RWDS}\ntrain: images\nval: images\nnames:\n  0: building")
    if len(os.listdir(os.path.join(OUT_RWDS, "images"))) > 100: return yaml_p
    jsons = ["Flooding_Group_combined_train_512_02.json", "Flooding_Group_India_train_512_02.json", "Flooding_Group_US_train_512_02.json"]
    for j in jsons:
        p = os.path.join(RWDS_ROOT, j)
        if not os.path.exists(p): continue
        with open(p, 'r') as f: data = json.load(f)
        imgs = {i['id']: i for i in data['images']}
        for a in tqdm(data['annotations'], desc=j):
            im = imgs.get(a['image_id'])
            if not im: continue
            fn = im['file_name']
            src = os.path.join(RWDS_ROOT, fn)
            if not os.path.exists(src):
                for r, _, fs in os.walk(RWDS_ROOT):
                    if fn in fs: src = os.path.join(r, fn); break
            if os.path.exists(src):
                dst = os.path.join(OUT_RWDS, "images", fn)
                if not os.path.exists(dst): shutil.copy2(src, dst)
                h, w = im['height'], im['width']
                x, y, bw, bh = a['bbox']
                with open(os.path.join(OUT_RWDS, "labels", os.path.splitext(fn)[0] + ".txt"), "a") as f:
                    f.write(f"0 {(x+bw/2)/w:.6f} {(y+bh/2)/h:.6f} {bw/w:.6f} {bh/h:.6f}\n")
    return yaml_p

rwds_cfg = prep_rwds()
results = []

if not os.path.exists(RUNS_DIR):
    print("Runs directory not found.")
else:
    # scan
    models = []
    for d in os.listdir(RUNS_DIR):
        w = os.path.join(RUNS_DIR, d, "weights", "best.pt")
        if os.path.exists(w): models.append((d, w))
    
    print(f"Found {len(models)} models.")
    for m_name, w in models:
        print(f"Testing {m_name}...")
        try:
            model = YOLO(w)
            r_orig = model.val(data=REAL_VAL, split='val', batch=32, device=0, half=True, plots=False, verbose=False)
            r_rwds = model.val(data=rwds_cfg, split='val', batch=32, device=0, half=True, plots=False, verbose=False)
            results.append({
                "Model": m_name,
                "Orig_mAP50": r_orig.box.map50, "Orig_mAP50-95": r_orig.box.map, "Orig_P": r_orig.box.mp, "Orig_R": r_orig.box.mr,
                "RWDS_mAP50": r_rwds.box.map50, "RWDS_mAP50-95": r_rwds.box.map, "RWDS_P": r_rwds.box.mp, "RWDS_R": r_rwds.box.mr
            })
        except Exception as e: print(f"Fail {m_name}: {e}")
        finally: del model; gc.collect(); torch.cuda.empty_cache()

if results:
    df = pd.DataFrame(results)
    df.to_excel(EXCEL_PATH, index=False)
    print(f"Saved to {EXCEL_PATH}")
    print(df)